In [17]:
from datasets import load_dataset

dataset = load_dataset("skeskinen/TinyStories-Instruct-hf")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2476533 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25028 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/diwank/.cache/huggingface/datasets/skeskinen___parquet/skeskinen--TinyStories-Instruct-hf-1f9111cb77858404/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
dataset = dataset["train"]

In [73]:
import re

def parse(row):
    text = row["text"]
    preface, story, *_ = text.split("Story:")

    preface = preface.strip()
    story = story.strip()
    
    lines = preface.split("\n")

    attributes = []

    for line in lines:
        k, *v = line.split(':')
        k = k.strip()
        v = ':'.join(v).strip()
        
        if not (k and v):
            continue

        attributes.append((k, v))

    return dict(preface=preface, story=story, attributes=attributes)

dataset = dataset.map(parse)

Map:   0%|          | 0/2476533 [00:00<?, ? examples/s]

In [78]:
len(dataset), len(dataset.filter(lambda row: len(set([k for k, v in row["attributes"]]) - {"Features", "Summary", "Words", "Random sentence"}) > 0))

Filter:   0%|          | 0/2476533 [00:00<?, ? examples/s]

(2476533, 52)

In [79]:
filtered_dataset = dataset.filter(lambda row: len(set([k for k, v in row["attributes"]]) - {"Features", "Summary", "Words", "Random sentence"}) > 0)

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/skeskinen___parquet/skeskinen--TinyStories-Instruct-hf-1f9111cb77858404/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c3285fccdaaaf0fe.arrow


In [80]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

In [81]:
filtered_dataset

Dataset({
    features: ['text', 'preface', 'story', 'attributes'],
    num_rows: 52
})

In [82]:
import re

def to_chatml(row):
    preface = row["preface"]
    story = row["story"]
    
    # Turn into chatml
    chatml = [
        situation("Theodore is talking to Samantha and intrigued by Samantha's storytelling."),
        person(f"Hey Samantha! Can you write a story that uses all of the following attributes:\n\n{preface}", name="Theodore"),
        me(f"Sure! Okay, let me think... Alright, how about this?\n\n{story}", name="Samantha")
    ]

    return dict(chatml=chatml)

filtered_dataset = filtered_dataset.map(to_chatml)

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

In [83]:
filtered_dataset.push_to_hub("diwank/tinystories-chatml", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]